<a href="https://colab.research.google.com/github/Junhojuno/Tensorflow-2.0-tutorial/blob/master/01_CNN_2_0style.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [0]:
from __future__ import absolute_import, division, print_function, unicode_literals

!pip install -q tensorflow-gpu==2.0.0-beta1
import tensorflow as tf

from tensorflow.keras import datasets, layers, models, Model

In [3]:
(train_images, train_labels), (test_images, test_labels) = datasets.mnist.load_data()

train_images = train_images[..., tf.newaxis] # train_images = train_images.reshape((60000, 28, 28, 1))와 같은 의미
test_images = test_images[..., tf.newaxis] # test_images = test_images.reshape((10000, 28, 28, 1))와 같은 의미


# 픽셀 값을 0~1 사이로 정규화합니다.
train_images, test_images = train_images / 255.0, test_images / 255.0
print("train image shape : {} \n test images shape : {}".format(train_images.shape, test_images.shape))

train image shape : (60000, 28, 28, 1) 
 test images shape : (10000, 28, 28, 1)


In [0]:
# tf.data를 활용하여 data slicing

train_ds = tf.data.Dataset.from_tensor_slices((train_images, train_labels)).shuffle(10000).batch(32)
test_ds = tf.data.Dataset.from_tensor_slices((test_images, test_labels)).batch(32)

In [0]:
# tf.keras로 subclassing

class CNN(Model):
    def __init__(self):
        super(CNN, self).__init__()
        self.conv1 = layers.Conv2D(filters=32, kernel_size=(3,3), activation='relu')
        self.maxpool1 = layers.MaxPooling2D((2,2))
        self.conv2 = layers.Conv2D(64, (3,3), activation='relu')
        self.maxpool2 = layers.MaxPooling2D((2,2))
        self.conv3 = layers.Conv2D(64, (3,3), activation='relu')
        self.flatten = layers.Flatten()
        self.dense1 = layers.Dense(64, activation='relu')
        self.dense2 = layers.Dense(10, activation='sigmoid')
        
    def call(self, x):
        x = self.conv1(x)
        x = self.maxpool1(x)
        x = self.conv2(x)
        x = self.maxpool2(x)
        x = self.conv3(x)
        x = self.flatten(x)
        x = self.dense1(x)
        return self.dense2(x)

model = CNN()

In [0]:
# optimizer, loss 설정
# tf.keras.losses.SparseCategoricalCrossentropy() : [[1],[3],[0],.....]와 같이 one-hot encoding이 안 된 상태일때 사용
# https://www.reddit.com/r/MLQuestions/comments/93ovkw/what_is_sparse_categorical_crossentropy/

loss_object = tf.keras.losses.SparseCategoricalCrossentropy()
optimizer = tf.keras.optimizers.Adam()


train_loss = tf.keras.metrics.Mean(name='Training loss')
train_acc = tf.keras.metrics.SparseCategoricalAccuracy(name='Training accuracy')

test_loss = tf.keras.metrics.Mean(name='Test loss')
test_acc = tf.keras.metrics.SparseCategoricalAccuracy(name='Test accuracy')

In [0]:
# tf.GradientTape() : Record operations for automatic differentiation.

# apply_gradient : Apply gradients to variables.
# This is the second part of `minimize()`. It returns an `Operation` that applies gradient

@tf.function
def training_step(images, labels):
    with tf.GradientTape() as g:
        prediction = model(images)
        loss = loss_object(labels, prediction)
    
    gradient = g.gradient(loss, model.trainable_variables)
    optimizer.apply_gradients(zip(gradient, model.trainable_variables))
    
    train_loss(loss) # 배치 사이즈 loss 평균때리기?
    train_acc(labels, prediction)
    
@tf.function
def test_step(images, labels):
    prediction = model(images)
    loss = loss_object(labels, prediction)
    
    test_loss(loss)
    test_acc(labels, prediction)

In [0]:
import time

In [19]:
epochs = 5

start = time.time()  # 시작 시간 저장 
 
for epoch in range(epochs):
    for image, label in train_ds:
        training_step(image, label)
        
    for image, label in test_ds:
        test_step(image, label)
        
    template = 'Epoch {}, Loss: {}, Accuracy: {}, Test Loss: {}, Test Accuracy: {}'
    print(template.format(epoch+1,                   
                          train_loss.result(),
                          train_acc.result()*100,
                          test_loss.result(),
                          test_acc.result()*100))
    
print("time :", time.time() - start)  # 현재시각 - 시작시간 = 실행 시간

Epoch 1, Loss: 0.11205559968948364, Accuracy: 96.52749633789062, Test Loss: 0.05645560473203659, Test Accuracy: 98.1050033569336
Epoch 2, Loss: 0.08766008913516998, Accuracy: 97.28055572509766, Test Loss: 0.04987150430679321, Test Accuracy: 98.36000061035156
Epoch 3, Loss: 0.0729563906788826, Accuracy: 97.73500061035156, Test Loss: 0.04925883561372757, Test Accuracy: 98.41999816894531
Epoch 4, Loss: 0.06289923936128616, Accuracy: 98.0469970703125, Test Loss: 0.0470350943505764, Test Accuracy: 98.52999877929688
Epoch 5, Loss: 0.05531470850110054, Accuracy: 98.27777862548828, Test Loss: 0.04474099352955818, Test Accuracy: 98.61500549316406
time : 68.07280421257019
